In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, classification_report
import lightgbm as lgb

In [2]:
from utils import *

In [3]:
df = load()
df.head()

,id,Airline,Flight,AirportFrom,AirportTo,DayOfWeek,Time,Length,Delay,Airline_DelayRate,Route,Route_AvgDelay
0,0,CO,269,SFO,IAH,3,15,205,1,0.566199,SFO_IAH,0.800866
1,1,US,1558,PHX,CLT,3,15,222,1,0.335971,PHX_CLT,0.418972
2,2,AA,2400,LAX,DFW,3,20,165,1,0.388470,LAX_DFW,0.347426
3,3,AA,2466,SFO,DFW,3,20,195,1,0.388470,SFO_DFW,0.525836
4,4,AS,108,ANC,SEA,3,30,202,0,0.339290,ANC_SEA,0.344519


In [4]:
for col in ['Airline', 'AirportFrom', 'AirportTo', 'Route', 'DayOfWeek']:
    df[col] = df[col].astype('category')


X_train, X_val, X_test, y_train, y_val, y_test = get_split_val(df)

In [5]:
model = LGBMClassifier(
    n_estimators=1000,
    learning_rate=0.01,
    num_leaves=63,
    random_state=0,
    n_jobs=-1,   
)

In [6]:
model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    categorical_feature=['Airline', 'AirportFrom', 'AirportTo', 'Route', 'DayOfWeek'],
    callbacks=[
        lgb.early_stopping(stopping_rounds=100, verbose=False),
        lgb.log_evaluation(period=100)
    ],
)
y_pred = model.predict(X_test)
print_metrics(y_test, y_pred)

[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Number of positive: 144158, number of negative: 179471
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000910 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5379
[LightGBM] [Info] Number of data points in the train set: 323629, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.445442 -> initscore=-0.219104
[LightGBM] [Info] Start training from score -0.219104
[100]	valid_0's binary_logloss: 0.624392
[200]	valid_0's binary_logloss: 0.612084
[300]	valid_0's binary_logloss: 0.609539
[400]	valid_0's binary_logloss: 0.609322
Accuracy: 0.6633758817912994

Classification